In [1]:
import pandas as pd
import pickle

In [2]:
import os

In [3]:
def boo(x):
    with(open('./dicts/'+x, 'rb')) as f:
        return pickle.load(f)

In [4]:
ld=[boo(i) for i in os.listdir('./dicts') if os.stat('./dicts/'+i).st_size>100]

In [5]:
df=pd.read_csv('start_page_2010.csv')

In [6]:
ids=df['link'].to_dict()

In [7]:
def foo(x):
    paths=[]
    values=[] 
    for i,j in x.items():
        if isinstance(j, list):
            path,value=i,j
        else:
            path, value=foo(j)
            path=[i]+path
        paths.append(path)
        values.append(value)
    return paths, values

In [8]:
def doo(qqq):
    x,y=foo(qqq)
    x,y=pd.DataFrame(x),pd.DataFrame(y)

    y.columns=range(1,y.columns.shape[0]+1)

    d2=y.lookup(y.index.tolist()*y.shape[1], [j for i in y.columns for j in [i]*y.shape[0]])
    d1=x.lookup(x.index.tolist()*y.shape[1], [j for i in y.columns for j in [i]*x.shape[0]])
    d0=x.lookup(x.index.tolist()*y.shape[1], [j for i in y.columns for j in [0]*x.shape[0]])

    data=pd.DataFrame([d0,d1,d2]).T.set_index(0).dropna().explode(2).explode(2).reset_index()
    #data[1]=data[1].map(lambda x: list(set(x)) if isinstance(x, list) else x)
    
    return data

In [9]:
output=pd.concat(list(map(doo, ld[:-1])))

AttributeError: 'list' object has no attribute 'items'

In [13]:
output.columns=['link','location','link_to_UIK']

In [24]:
output.shape

(6846, 3)

In [25]:
merged_df=df.merge(output, on='link')

In [26]:
merged_df.to_csv('preliminary_results.csv', index=False)

In [32]:
print(merged_df.sample().link_to_UIK.iloc[0])

http://www.yakut.vybory.izbirkom.ru/region/yakut?action=show&root=144027001&tvd=4144027114850&vrn=414402788176&prver=0&pronetvd=null&region=14&sub_region=14&type=0&vibid=4144027114850
